In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input, Dropout, Activation, Dense, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [8]:
filepath = "C:/Users/gufra/OneDrive/Desktop/Detection-TonIoT23-main/UNSW_2018_IoT_Botnet_Full5pc_4.csv"
training_dataset = pd.read_csv(filepath)

In [9]:
training_dataset

,pkSeqID,stime,flgs,flgs_number,proto,proto_number,saddr,sport,daddr,dport,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,3000001,1.528099e+09,e,1,udp,3,192.168.100.147,6226,192.168.100.3,80,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
1,3000002,1.528099e+09,e,1,udp,3,192.168.100.147,6227,192.168.100.3,80,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
2,3000003,1.528099e+09,e,1,udp,3,192.168.100.147,6228,192.168.100.3,80,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
3,3000004,1.528099e+09,e,1,udp,3,192.168.100.147,6229,192.168.100.3,80,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
4,3000005,1.528099e+09,e,1,udp,3,192.168.100.147,6230,192.168.100.3,80,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668517,3668518,1.529381e+09,e,1,tcp,1,192.168.100.150,35064,192.168.100.3,22,...,9.889330,19,19,455.754000,9.889330,30,30,1,Theft,Keylogging
668518,3668519,1.529381e+09,e,1,tcp,1,192.168.100.150,35066,192.168.100.3,22,...,9.889330,19,19,10453.000000,9.889330,30,30,1,Theft,Keylogging
668519,3668520,1.529381e+09,e,1,tcp,1,192.168.100.150,35070,192.168.100.3,22,...,9.889330,19,19,10.785200,9.889330,441,441,1,Theft,Keylogging
668520,3668521,1.529381e+09,e,1,tcp,1,192.168.100.3,43001,192.168.100.150,4433,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,Theft,Keylogging


In [10]:
#dropping columns
training_dataset=training_dataset.drop(['stime'],axis=1)
training_dataset=training_dataset.drop(['flgs'],axis=1)
training_dataset=training_dataset.drop(['saddr'],axis=1)
training_dataset=training_dataset.drop(['daddr'],axis=1)
training_dataset=training_dataset.drop(['ltime'],axis=1)
training_dataset=training_dataset.drop(['state'],axis=1)

In [11]:
training_dataset

,pkSeqID,flgs_number,proto,proto_number,sport,dport,pkts,bytes,state_number,seq,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,3000001,1,udp,3,6226,80,15,900,4,109223,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
1,3000002,1,udp,3,6227,80,15,900,4,109224,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
2,3000003,1,udp,3,6228,80,15,900,4,109225,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
3,3000004,1,udp,3,6229,80,15,900,4,109226,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
4,3000005,1,udp,3,6230,80,15,900,4,109227,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,DDoS,UDP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668517,3668518,1,tcp,1,35064,22,6,434,1,81,...,9.889330,19,19,455.754000,9.889330,30,30,1,Theft,Keylogging
668518,3668519,1,tcp,1,35066,22,6,434,1,82,...,9.889330,19,19,10453.000000,9.889330,30,30,1,Theft,Keylogging
668519,3668520,1,tcp,1,35070,22,31,5472,6,84,...,9.889330,19,19,10.785200,9.889330,441,441,1,Theft,Keylogging
668520,3668521,1,tcp,1,43001,4433,2,134,1,85,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,Theft,Keylogging


In [12]:
#replacing values
training_dataset['proto']=training_dataset['proto'].replace({'udp':'0','tcp':'1','icmp':'2','arp':'3','ipv6-icmp':'4'})
training_dataset['category']=training_dataset['category'].replace({'DDoS':'0','Normal':'1','Reconnaissance':'2','Theft':'3'})
training_dataset['subcategory']=training_dataset['subcategory'].replace({'UDP':'0','Normal':'1','OS_Fingerprint':'2','Service_Scan':'3','Data_Exfiltration':'4','Keylogging':'5'})


In [13]:
training_dataset

,pkSeqID,flgs_number,proto,proto_number,sport,dport,pkts,bytes,state_number,seq,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,3000001,1,0,3,6226,80,15,900,4,109223,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
1,3000002,1,0,3,6227,80,15,900,4,109224,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
2,3000003,1,0,3,6228,80,15,900,4,109225,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
3,3000004,1,0,3,6229,80,15,900,4,109226,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
4,3000005,1,0,3,6230,80,15,900,4,109227,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668517,3668518,1,1,1,35064,22,6,434,1,81,...,9.889330,19,19,455.754000,9.889330,30,30,1,3,5
668518,3668519,1,1,1,35066,22,6,434,1,82,...,9.889330,19,19,10453.000000,9.889330,30,30,1,3,5
668519,3668520,1,1,1,35070,22,31,5472,6,84,...,9.889330,19,19,10.785200,9.889330,441,441,1,3,5
668520,3668521,1,1,1,43001,4433,2,134,1,85,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,3,5


In [14]:
#handling the infinite value
training_dataset.replace([np.inf,-np.inf],np.nan,inplace=True)
training_dataset.fillna(training_dataset.mean(),inplace=True)
training_dataset

,pkSeqID,flgs_number,proto,proto_number,sport,dport,pkts,bytes,state_number,seq,...,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,attack,category,subcategory
0,3000001,1,0,3,6226,80,15,900,4,109223,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
1,3000002,1,0,3,6227,80,15,900,4,109224,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
2,3000003,1,0,3,6228,80,15,900,4,109225,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
3,3000004,1,0,3,6229,80,15,900,4,109226,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
4,3000005,1,0,3,6230,80,15,900,4,109227,...,1.098250,100,100,1.098270,1.098250,1500,1500,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668517,3668518,1,1,1,35064,22,6,434,1,81,...,9.889330,19,19,455.754000,9.889330,30,30,1,3,5
668518,3668519,1,1,1,35066,22,6,434,1,82,...,9.889330,19,19,10453.000000,9.889330,30,30,1,3,5
668519,3668520,1,1,1,35070,22,31,5472,6,84,...,9.889330,19,19,10.785200,9.889330,441,441,1,3,5
668520,3668521,1,1,1,43001,4433,2,134,1,85,...,666667.000000,1,3,666667.000000,22346.400000,2,4,1,3,5


In [15]:
y = training_dataset['attack']
x=training_dataset.drop(['attack'],axis=1)
x

,pkSeqID,flgs_number,proto,proto_number,sport,dport,pkts,bytes,state_number,seq,...,AR_P_Proto_P_SrcIP,AR_P_Proto_P_DstIP,N_IN_Conn_P_DstIP,N_IN_Conn_P_SrcIP,AR_P_Proto_P_Sport,AR_P_Proto_P_Dport,Pkts_P_State_P_Protocol_P_DestIP,Pkts_P_State_P_Protocol_P_SrcIP,category,subcategory
0,3000001,1,0,3,6226,80,15,900,4,109223,...,1.098250,1.098250,100,100,1.098270,1.098250,1500,1500,0,0
1,3000002,1,0,3,6227,80,15,900,4,109224,...,1.098250,1.098250,100,100,1.098270,1.098250,1500,1500,0,0
2,3000003,1,0,3,6228,80,15,900,4,109225,...,1.098250,1.098250,100,100,1.098270,1.098250,1500,1500,0,0
3,3000004,1,0,3,6229,80,15,900,4,109226,...,1.098250,1.098250,100,100,1.098270,1.098250,1500,1500,0,0
4,3000005,1,0,3,6230,80,15,900,4,109227,...,1.098250,1.098250,100,100,1.098270,1.098250,1500,1500,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668517,3668518,1,1,1,35064,22,6,434,1,81,...,9.889330,9.889330,19,19,455.754000,9.889330,30,30,3,5
668518,3668519,1,1,1,35066,22,6,434,1,82,...,9.889330,9.889330,19,19,10453.000000,9.889330,30,30,3,5
668519,3668520,1,1,1,35070,22,31,5472,6,84,...,9.889330,9.889330,19,19,10.785200,9.889330,441,441,3,5
668520,3668521,1,1,1,43001,4433,2,134,1,85,...,22346.400000,666667.000000,1,3,666667.000000,22346.400000,2,4,3,5


In [16]:
y

0         1
1         1
2         1
3         1
4         1
         ..
668517    1
668518    1
668519    1
668520    1
668521    1
Name: attack, Length: 668522, dtype: int64

In [17]:
scaler = MinMaxScaler()

In [18]:
scaler.fit(x)

MinMaxScaler()

In [19]:
normalized_x = scaler.transform(x)

In [20]:
normalized_x

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.27100680e-02, 0.00000000e+00, 0.00000000e+00],
       [1.49583932e-06, 0.00000000e+00, 0.00000000e+00, ...,
        1.27100680e-02, 0.00000000e+00, 0.00000000e+00],
       [2.99167865e-06, 0.00000000e+00, 0.00000000e+00, ...,
        1.27100680e-02, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.99997008e-01, 0.00000000e+00, 2.50000000e-01, ...,
        3.73077380e-03, 1.00000000e+00, 1.00000000e+00],
       [9.99998504e-01, 0.00000000e+00, 2.50000000e-01, ...,
        2.54370941e-05, 1.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 7.50000000e-01, ...,
        1.27185470e-04, 1.00000000e+00, 1.00000000e+00]])

In [21]:
normalized_x.shape

(668522, 39)

In [22]:
y = y.array.reshape(-1,1)

In [23]:
scaler.fit(y)

MinMaxScaler()

In [24]:
normalized_y = scaler.transform(y)

In [25]:
normalized_y

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [28]:
X_train.shape

(534817, 39)

In [29]:
X_test.shape

(133705, 39)

In [79]:

model = Sequential()
model.add(Dense(2000, activation='relu',input_dim=(39)))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(800,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [80]:
model.input_shape

(None, 39)

In [81]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_68 (Dense)            (None, 2000)              80000     
                                                                 
 dense_69 (Dense)            (None, 1500)              3001500   
                                                                 
 dropout_44 (Dropout)        (None, 1500)              0         
                                                                 
 dense_70 (Dense)            (None, 800)               1200800   
                                                                 
 dropout_45 (Dropout)        (None, 800)               0         
                                                                 
 dense_71 (Dense)            (None, 400)               320400    
                                                                 
 dropout_46 (Dropout)        (None, 400)             

In [82]:
import time

In [84]:
start = time.time()
print('program start...')
print()

history = model.fit(X_train, Y_train, epochs = 2, batch_size=256, validation_data=(X_test,Y_test),verbose=1)

print()
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...

Epoch 1/2
2090/2090 [==============================] - 276s 132ms/step - loss: 1.8568e-04 - accuracy: 1.0000 - val_loss: 1.2992e-04 - val_accuracy: 0.9999
Epoch 2/2
2090/2090 [==============================] - 280s 134ms/step - loss: 1.3298e-04 - accuracy: 1.0000 - val_loss: 3.0557e-07 - val_accuracy: 1.0000

program end...

time cost: 
556.203173160553 seconds
